In [ ]:
mport pandas as pd
import numpy as np
from torch import Tensor
import torch.nn as nn
from transformers import AutoTokenizer, BatchEncoding, PreTrainedTokenizerBase, AutoModel
from transformers import BertModel
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
import torch
import math
import os
from typing import Any,  Union
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/MyDrive/dataset/")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "expandable_segments:True"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Mi sposto nella cartella per importare il file
%cd /content/drive/MyDrive/dataset/

#Importo il file tramite pandas e pulisco il contenuto text da caratteri di escape
train = pd.read_csv("reviews_dataset.csv", index_col='id')

#Pulisco le recensioni nella colonna text da tutti i caratteri di escape come \n
train['text'] = train['text'].replace( r'\\[abfnrtv\\"]', ' ', regex=True )
#Pulisco le recensioni nella colonna text da tutti i caratteri non alfanumerici
train['text'] = train['text'].replace( r'\W', ' ', regex=True )


train['len'] = train['text'].str.len()
max_len = train['len'].max()

n_star = 5

dev_frac = 0.1
test_frac = 0.1
train_frac = (1 - dev_frac - test_frac)

#Mischio il dataframe (estraggo in maniera casuale tutte le righe del DataFrame)
both = train.sample(frac=1/20)

#Creo i dataset per dev-train-set
train_1 = both.sample(frac=train_frac)
test = both.sample(frac=test_frac)
dev = both.sample(frac=dev_frac)
train.head()

/content/drive/MyDrive/dataset


,text,label,len
id,,,
0,No sirve para la calle Tiene el gran problema...,1,614
1,No me ha llegado No me ha llegado Estoy espe...,0,76
2,Mal servicio Producto no recibido Aunque la...,0,160
3,Mala experiencia Muy disgustado con el produc...,0,138
4,Luz bicicleta Es muy buena luz y dura bastant...,4,59


In [ ]:
tokenize = AutoTokenizer.from_pretrained("bert-base-cased" )

def transform_list_of_texts(
    texts: list[str],
    tokenizer: PreTrainedTokenizerBase,
    chunk_size: int,
    stride: int,
    minimal_chunk_length: int
) -> BatchEncoding:
    model_inputs = [
        transform_single_text(text, tokenizer, chunk_size, stride, minimal_chunk_length)
        for text in texts
    ]
    input_ids = [model_input[0] for model_input in model_inputs]
    attention_mask = [model_input[1] for model_input in model_inputs]
    tokens = {"input_ids": input_ids, "attention_mask": attention_mask}
    return BatchEncoding(tokens)


def transform_single_text(
    text: str,
    tokenizer: PreTrainedTokenizerBase,
    chunk_size: int,
    stride: int,
    minimal_chunk_length: int,
) -> tuple[Tensor, Tensor]:
    """Transforms (the entire) text to model input of BERT model."""
    tokens = tokenize_whole_text(text, tokenizer)
    input_id_chunks, mask_chunks = split_tokens_into_smaller_chunks(tokens, chunk_size, stride, minimal_chunk_length)
    add_special_tokens_at_beginning_and_end(input_id_chunks, mask_chunks)
    add_padding_tokens(input_id_chunks, mask_chunks)
    input_ids, attention_mask = stack_tokens_from_all_chunks(input_id_chunks, mask_chunks)
    return input_ids, attention_mask


def tokenize_whole_text(text: str, tokenizer: PreTrainedTokenizerBase) -> BatchEncoding:
    """Tokenizes the entire text without truncation and without special tokens."""
    tokens = tokenizer(text, add_special_tokens=False, truncation=False, return_tensors="pt")
    return tokens


def tokenize_text_with_truncation(
    text: str, tokenizer: PreTrainedTokenizerBase, maximal_text_length: int
) -> BatchEncoding:
    """Tokenizes the text with truncation to maximal_text_length and without special tokens."""
    tokens = tokenizer(
        text, add_special_tokens=False, max_length=maximal_text_length, truncation=True, return_tensors="pt"
    )
    return tokens


def split_tokens_into_smaller_chunks(
    tokens: BatchEncoding,
    chunk_size: int,
    stride: int,
    minimal_chunk_length: int,
) -> tuple[list[Tensor], list[Tensor]]:
    """Splits tokens into overlapping chunks with given size and stride."""
    input_id_chunks = split_overlapping(tokens["input_ids"][0], chunk_size, stride, minimal_chunk_length)
    mask_chunks = split_overlapping(tokens["attention_mask"][0], chunk_size, stride, minimal_chunk_length)
    return input_id_chunks, mask_chunks


def add_special_tokens_at_beginning_and_end(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    """
    Adds special CLS token (token id = 101) at the beginning.
    Adds SEP token (token id = 102) at the end of each chunk.
    Adds corresponding attention masks equal to 1 (attention mask is boolean).
    """
    if len(input_id_chunks) == 0:
        input_id_chunks.append(torch.Tensor([101, 102]))
        mask_chunks.append(torch.Tensor([1, 1]))
        return
    for i in range(len(input_id_chunks)):
        # adding CLS (token id 101) and SEP (token id 102) tokens
        input_id_chunks[i] = torch.cat([Tensor([101]), input_id_chunks[i], Tensor([102])])
        # adding attention masks  corresponding to special tokens
        mask_chunks[i] = torch.cat([Tensor([1]), mask_chunks[i], Tensor([1])])


def add_padding_tokens(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> None:
    """Adds padding tokens (token id = 0) at the end to make sure that all chunks have exactly 512 tokens."""
    for i in range(len(input_id_chunks)):
        # get required padding length
        pad_len = 512 - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([input_id_chunks[i], Tensor([0] * pad_len)])
            mask_chunks[i] = torch.cat([mask_chunks[i], Tensor([0] * pad_len)])


def stack_tokens_from_all_chunks(input_id_chunks: list[Tensor], mask_chunks: list[Tensor]) -> tuple[Tensor, Tensor]:
    """Reshapes data to a form compatible with BERT model input."""
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    return input_ids.long(), attention_mask.int()


def split_overlapping(tensor: Tensor, chunk_size: int, stride: int, minimal_chunk_length: int) -> list[Tensor]:
    """Helper function for dividing 1-dimensional tensors into overlapping chunks."""
    result = [tensor[i : i + chunk_size] for i in range(0, len(tensor), stride)]
    if len(result) > 1:
        # ignore chunks with less than minimal_length number of tokens
        result = [x for x in result if len(x) >= minimal_chunk_length]
    return result



In [ ]:
train_tok = transform_list_of_texts(train_1['text'].to_list(), tokenize, 510, 510, 510 )
test_tok = transform_list_of_texts(test['text'].to_list(), tokenize, 510, 510, 510 )

class TokenizedDataset(Dataset):
    """Dataset for tokens with optional labels."""

    def __init__(self, tokens: BatchEncoding, labels: list):
        self.input_ids = tokens["input_ids"]
        self.attention_mask = [att > 0  for att in tokens["attention_mask"]]
        self.labels = labels

    def __len__(self) -> int:
        return len(self.input_ids)

    def __getitem__(self, idx: int) -> Union[tuple[Tensor, Tensor, Any], tuple[Tensor, Tensor]]:
        if self.labels:
            return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]
        return self.input_ids[idx], self.attention_mask[idx]

train_dataset = TokenizedDataset(train_tok, train_1['label'].to_list())
test_dataset = TokenizedDataset(test_tok, test['label'].to_list())


Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
vocab = tokenize.vocab_size
print(vocab)

28996


In [ ]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)

test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, accumulation_steps):

  model.train()
  size = 0
  correct = 0
  for step, batch in enumerate(dataloader):
    input_tokens, label = batch[0].to(device), batch[-1]
    attention_mask = batch[1].to(device)
    number_of_chunks = [len(x) for x in input_tokens]
    one_hot_label = nn.functional.one_hot(label, num_classes=n_star).to(float).to(device)
    # concateno tutti i token di input in uno singolo
    input_tokens_combined = []
    for x in input_tokens:
        input_tokens_combined.extend(x.tolist())

    input_tokens_combined_tensors = torch.stack([torch.tensor(x).to(device) for x in input_tokens_combined])

    # concateno tutte le maschere di Padding per il meccanismo dell'attenzione
    attention_mask_combined = []
    for x in attention_mask:
        attention_mask_combined.extend(x.tolist())

    attention_mask_combined_tensors = torch.stack(
        [torch.tensor(x).to(device) for x in attention_mask_combined]
    )

    # ottengo le probabilità dall'encoder del transformer
    logits = model(input_tokens_combined_tensors, attention_mask_combined_tensors)

    # divido in pezzi i rrisultati per calcolarne infine la media finale
    logits_split = logits.split(number_of_chunks, dim=0)

    pooled_logits = torch.stack([torch.mean(x, dim=0) for x in logits_split])
    probs = nn.Softmax(1)(pooled_logits)
    #Calcolo gli one hot label dalle etichette per il calcolo della funzione di loss
    loss = loss_fn(probs, one_hot_label)

    #Backpropagation
    loss.backward()
    #Caolcolo l'accuratezza del singolo batch
    out = probs.argmax(1).to("cpu")
    correct += ( out == label).to(float).sum(0).item()
    size += 64
    curr_acc = correct/size
    #optimizer.step()
    if ((step + 1) % accumulation_steps == 0) or (step + 1 == len(dataloader)):
      optimizer.step()
      optimizer.zero_grad()


    if step % 100 == 0:
      loss= loss.item()

      print(f"Funzione di loss: {loss:>7f} Errore train: \n Accuratezza: {(100*curr_acc):>0.1f}%, step[{step}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = 0
    correct = 0
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
      for step, batch in enumerate(dataloader):
        input_tokens, label = batch[0].to(device), batch[-1]
        attention_mask = batch[1].to(device)
        number_of_chunks = [len(x) for x in input_tokens]
        one_hot_label = nn.functional.one_hot(label, num_classes=n_star).to(float).to(device)
        # concateno tutti i token di input in uno singolo
        input_tokens_combined = []
        for x in input_tokens:
            input_tokens_combined.extend(x.tolist())

        input_tokens_combined_tensors = torch.stack([torch.tensor(x).to(device) for x in input_tokens_combined])

        # concateno tutte le maschere di Padding per il meccanismo dell'attenzione
        attention_mask_combined = []
        for x in attention_mask:
            attention_mask_combined.extend(x.tolist())

        attention_mask_combined_tensors = torch.stack(
            [torch.tensor(x).to(device) for x in attention_mask_combined]
        )

        # ottengo le probabilità dall'encoder del transformer
        logits = model(input_tokens_combined_tensors, attention_mask_combined_tensors)

        # divido in pezzi i rrisultati per calcolarne infine la media finale
        logits_split = logits.split(number_of_chunks, dim=0)

        pooled_logits = torch.stack([torch.mean(x, dim=0) for x in logits_split])
        probs = nn.Softmax(1)(pooled_logits)
        #Calcolo gli one hot label dalle etichette per il calcolo della funzione di loss
        loss = loss_fn(probs, one_hot_label)

        test_loss += loss.item()
        #Caolcolo l'accuratezza del singolo batch
        out = probs.argmax(1).to("cpu")
        correct += ( out == label).to(float).sum(0).item()
    size = len(test['text'])

    test_loss /= num_batches
    correct /= size
    print(f"Errore test: \n Accuratezza: {(100*correct):>0.1f}%, Media loss: {test_loss:>8f} \n")

In [ ]:
class BertNN(nn.Module):
    def __init__(self, model: BertModel, num_labels: int):
        super().__init__()
        self.model = model

        # classification head
        self.fin_seq = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(64),
            nn.Linear(64,num_labels)
        )

    def forward(self, input_ids: Tensor, attention_mask: Tensor) -> Tensor:
        x = self.model(input_ids, attention_mask)
        x = x[0][:, 0, :]  # take <s> token (equiv. to [CLS])

        # classification head
        x = self.fin_seq(x)
        return x

bert = AutoModel.from_pretrained("bert-base-uncased")
model = BertNN(model=bert, num_labels=5)

for name, param in model.named_parameters():
    if "encoder.layer.9" in name or "encoder.layer.10" in name or "encoder.layer.11" in name or "fin_seq" in name:
        param.requires_grad = True  # Sblocca solo gli ultimi due layer
    else:
        param.requires_grad = False  # Congela gli altri

for _, param in model.fin_seq.named_parameters():
    if param.data.dim() == 2:
        nn.init.xavier_uniform_(param)


if torch.cuda.is_available() and torch.cuda.device_count():
  model = nn.DataParallel(model)


loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum = 0.7)
optimizer = torch.optim.AdamW([
    {"params": model.module.model.parameters(), "lr": 5e-6},  # Learning rate basso per BERT
    {"params": model.module.fin_seq.parameters(), "lr": 1e-1}    # Learning rate più alto per FC
])

In [ ]:
for epoch in range(4):
  print("Epoch: ", epoch)
  train_loop(train_dataloader, model, loss_fn, optimizer, 10)


test_loop(test_dataloader, model, loss_fn)

Epoch:  0
Funzione di loss: 1.653052 Errore train: 
 Accuratezza: 23.4%, step[0]
Funzione di loss: 1.623232 Errore train: 
 Accuratezza: 20.6%, step[100]
Funzione di loss: 1.627660 Errore train: 
 Accuratezza: 20.8%, step[200]
Epoch:  1
Funzione di loss: 1.618555 Errore train: 
 Accuratezza: 20.3%, step[0]
Funzione di loss: 1.596468 Errore train: 
 Accuratezza: 22.0%, step[100]
Funzione di loss: 1.614030 Errore train: 
 Accuratezza: 21.9%, step[200]
Epoch:  2
Funzione di loss: 1.604040 Errore train: 
 Accuratezza: 21.9%, step[0]
Funzione di loss: 1.590026 Errore train: 
 Accuratezza: 22.6%, step[100]
Funzione di loss: 1.571468 Errore train: 
 Accuratezza: 22.8%, step[200]
Epoch:  3
Funzione di loss: 1.635813 Errore train: 
 Accuratezza: 17.2%, step[0]
Funzione di loss: 1.588374 Errore train: 
 Accuratezza: 24.1%, step[100]
Funzione di loss: 1.592777 Errore train: 
 Accuratezza: 25.1%, step[200]
Errore test: 
 Accuratezza: 26.9%, Media loss: 1.577882 



In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/dataset/")